На примере решения заданий из урока "Методы сегментации клиентов и целевой аудитории" курса "Аналитик данных" karpov.courses

[Подготовка данных](#0)

[Знакомство с данными](#1)

[Какое максимальное кол-во покупок было совершено одним пользователем?](#2)

[Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М?](#3)

[Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F? ](#4)

[Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R? ](#5)

[Сколько пользователей попало в сегмент 111? ](#6)

[Сколько пользователей попало в сегмент 311? ](#7)

[В каком RFM-сегменте самое большое кол-во пользователей?](#8)

[В каком RFM-сегменте самое маленькое кол-во пользователей?](#9)

[Какое количество пользователей попало в самый малочисленный сегмент?](#10)
<a id="1"></a>


In [152]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=2, style="whitegrid", rc={'figure.figsize' : (15,7)})
import matplotlib.pyplot as plt
from scipy import special
plt.style.use('dark_background')

In [235]:
url='https://drive.google.com/file/d/1UoC_tl7Q_dZzhmCMTQAJc86vqpba-GI5/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, dtype = str)
df.head(1)

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.0


<a id="0"></a> Подготовка данных

In [191]:
df.dtypes

InvoiceNo       object
CustomerCode    object
InvoiceDate     object
Amount          object
dtype: object

In [236]:
df = df.rename(columns = {'InvoiceNo'    : 'invoice_no',
                          'CustomerCode' : 'customer_code',
                          'InvoiceDate'  : 'invoice_date',
                          'Amount'       : 'amount'})
df.amount = df.amount.astype(float)
df.invoice_date = pd.to_datetime(df.invoice_date)

In [193]:
df.dtypes

invoice_no               object
customer_code            object
invoice_date     datetime64[ns]
amount                  float64
dtype: object

<a id="1"></a> Знакомство с данными

In [194]:
df

,invoice_no,customer_code,invoice_date,amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00
...,...,...,...,...
332725,S0081810310459,14092500,2020-09-30,3801.87
332726,S0081810310461,99065678,2020-09-30,5769.88
332727,S0081810310462,19029918,2020-09-30,736.88
332728,S0081810310463,13020033,2020-09-30,1475.20


In [195]:
pd.DataFrame({'':['start date', 'end_date'],' ': [df['invoice_date'].min(),df['invoice_date'].max()]})

,,
0,start date,2020-09-01
1,end_date,2020-09-30


In [196]:
df.invoice_no.nunique() == len(df)

True

In [197]:
df.customer_code.nunique() == len(df)

False

In [203]:
df.customer_code.value_counts()

customer_code
19057820    204
13215452    113
13032521    106
19080880     99
99003061     90
           ... 
35103508      1
13242518      1
13020190      1
99031449      1
19054700      1
Name: count, Length: 123733, dtype: int64

<a id="2"></a> Какое максимальное кол-во покупок было совершено одним пользователем?

In [207]:
df.customer_code.value_counts().head(1)

customer_code
19057820    204
Name: count, dtype: int64

<a id="3"></a> Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? 
(Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)?

In [210]:
df.head(1)

,invoice_no,customer_code,invoice_date,amount
0,C0011810010001,19067290,2020-09-01,1716.0


In [237]:
df['revenue'] = df.amount
last_date = df.invoice_date.max()
last_date

Timestamp('2020-09-30 00:00:00')

In [238]:
df = df.groupby('customer_code').agg({'invoice_date'  : lambda x: (last_date - x.max()).days,
                                      'amount'        : lambda x: len(x),
                                      'revenue'       : lambda x: x.sum()})


In [239]:
df.columns = ['recency','frequency', 'monetary']


In [240]:
df = df.reset_index()
df

,customer_code,recency,frequency,monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


In [227]:
df.dtypes

customer_code     object
recency            int64
frequency          int64
monetary         float64
dtype: object

In [241]:
quantiles = df[['recency','frequency','monetary']].quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [234]:
quantiles.monetary[0.25]

np.float64(765.0)

<a id="4"></a> Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

In [244]:
quantiles.frequency[0.75] + 1

np.float64(4.0)

<a id="5"></a> Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [243]:
quantiles.recency[0.5]

np.float64(8.0)

<a id="6"></a> Сколько пользователей попало в сегмент 111?

In [229]:
def r (value):
    if value <= quantiles.recency[0.25]:
        return 1
    elif value <= quantiles.recency[0.5]:
        return 2
    elif value <= quantiles.recency[0.75]:
        return 3
    else:
        return 4
    
def fm (value, metric):
    if value <= quantiles[metric][0.25]:
        return 4
    elif value <= quantiles[metric][0.5]:
        return 3
    elif value <= quantiles[metric][0.75]:
        return 2
    else:
        return 1

In [230]:
df_rfm= df

In [231]:
df_rfm['r']= df['recency'].apply(lambda x: r(x))
df_rfm['f']= df['frequency'].apply(lambda x: fm(x, 'frequency'))
df_rfm['m']= df['monetary'].apply(lambda x: fm(x, 'monetary'))
df_rfm

,customer_code,recency,frequency,monetary,r,f,m
0,02213019,19,1,1609.20,4,4,3
1,02213042,22,3,9685.48,4,2,1
2,02213071,29,1,415.00,4,4,4
3,02213088,23,1,305.00,4,4,4
4,02213092,25,1,1412.88,4,4,3
...,...,...,...,...,...,...,...
123728,99099927,10,1,961.10,3,4,3
123729,99099936,0,1,1521.78,1,4,3
123730,99099959,8,2,1444.56,2,3,3
123731,99099963,19,1,3018.91,4,4,2


In [245]:
df_rfm['rfm'] = df_rfm.r.astype(str) + df_rfm.f.astype(str) + df_rfm.m.astype(str)
df_rfm[['customer_code','r','f', 'm', 'rfm']]

,customer_code,r,f,m,rfm
0,02213019,4,4,3,443
1,02213042,4,2,1,421
2,02213071,4,4,4,444
3,02213088,4,4,4,444
4,02213092,4,4,3,443
...,...,...,...,...,...
123728,99099927,3,4,3,343
123729,99099936,1,4,3,143
123730,99099959,2,3,3,233
123731,99099963,4,4,2,442


In [248]:
df_rfm[df_rfm.rfm == '111'].rfm.count()

np.int64(9705)

<a id="7"></a> Сколько пользователей попало в сегмент 311?

In [250]:
df_rfm[df_rfm.rfm == '311'].rfm.count()

np.int64(1609)

<a id="8"></a> В каком RFM-сегменте самое большое кол-во пользователей?

In [255]:
df_rfm.groupby('rfm').rfm.value_counts().idxmax()

'444'

<a id="9"></a> В каком RFM-сегменте самое маленькое кол-во пользователей?

In [256]:
df_rfm.groupby('rfm').rfm.value_counts().idxmin()

'414'

<a id="10"></a> Какое количество пользователей попало в самый малочисленный сегмент?

In [261]:
df_rfm.groupby('rfm').rfm.value_counts().sort_values().head(1).iloc[0]

np.int64(2)